# Single-Node Rendering From a Pandas Dataframe
This example will be fairly similar to the previous example with a Numpy array, except we'll load and visualize a star dataset. Just like that example, we'll start out by loading up the data.

In [ ]:
import pandas as pd
df = pd.read_csv("/data/Gaia/ds.gal.csv")

try:
    import PVGeo
except ImportError:
    print("Error: PVGeo isn't available in the current environment")
    
vtkt = PVGeo.interface.data_frame_to_table(df) # vtkTable object with our star data

# We also define the names of the columns we want to use for (x,y,z) position
posCols = ['x[pc]', 'y[pc]', 'z[pc]']

Unlike Numpy, Pandas isn't natively supported in VTK. Luckily, PVGeo has a tool to translate Pandas dataframes to vtkTables. From here, we can move on to setting up the ParaView state. Notice that we're using a more advanced pipeline this time. Once we have the TrivialProducer for the vtkTable, we have to convert it to a set of points, then create a 3D glyph for each point. We also color the points according to the 'plxmas' variable, and display the color legend in the visualization window. Also, since we're using a black body scale to color our stars, we set the background color to something non-black.

In [ ]:
from paraview.simple import *
paraview.simple._DisableFirstRenderCameraReset()

# wrap the vtkTable in a producer, then apply a TableToPoints filter
stars = TrivialProducer()
stars.GetClientSideObject().SetOutput(vtkt)
stars.UpdatePipeline()
ttp = TableToPoints(Input=stars)
ttp.XColumn = posCols[0]
ttp.YColumn = posCols[1]
ttp.ZColumn = posCols[2]

# Create a new 'Render View'
renV = CreateView('RenderView')
renV.ViewSize = [800, 500]
renV.CenterOfRotation = [-42.81136055702743, 20.561263821238143, -8.375310213352522]
renV.CameraFocalPoint = renV.CenterOfRotation
renV.CameraPosition = [-19.004408839051408, 78.59672562652541, -351.72329889073706]
renV.CameraViewUp = [0,1,0]
renV.Background = [0.32, 0.34, 0.43]
renV.EnableRayTracing = 1
renV.BackEnd = 'OptiX pathtracer'

# create the color lookup table that we'll use to set star color
plxmasLUT = GetColorTransferFunction('plxmas')
plxmasLUT.RGBPoints = [5.800048770935931, 0.0, 0.0, 0.0, 26.651891986128916, 0.901960784314, 0.0, 0.0, 47.5037352013219, 0.901960784314, 0.901960784314, 0.0, 57.92965680891839, 1.0, 1.0, 1.0]
plxmasLUT.ColorSpace = 'RGB'
plxmasLUT.NanColor = [0.0, 0.498039215686, 1.0]
plxmasLUT.ScalarRangeInitialized = 1.0

# create a Glyph filter to create a sphere for each star
glyph = Glyph(Input=ttp, GlyphType='Sphere')
glyph.ScaleFactor = 29.995732905481237
glyph.GlyphTransform = 'Transform2'
glyph.GlyphMode = 'All Points'
glyph.GlyphType.Radius = 0.025
glyph.GlyphType.ThetaResolution = 180
glyph.GlyphType.PhiResolution = 90
glyphDisplay = Show(glyph, renV)
glyphDisplay.Representation = 'Surface'
glyphDisplay.ColorArrayName = ['POINTS', 'plx[mas]']
glyphDisplay.LookupTable = plxmasLUT

# finally, show the color legend
plxmasLUTColorBar = GetScalarBar(plxmasLUT, renV)
plxmasLUTColorBar.Title = 'plx[mas]'
plxmasLUTColorBar.Visibility = 1

Then, just create a PVDisplay widget and pass it the desired RenderView

In [ ]:
from ipyparaview.widgets import PVDisplay
w = PVDisplay(renV)
w

If we change our minds about the variable we want to color our data by, we can just add an iPython widget to control that.

In [ ]:
from ipywidgets import interact

def setVar(v):
    ColorBy(glyphDisplay, ('POINTS', v))
    glyphDisplay.RescaleTransferFunctionToDataRange(True, False)
    vLUT = GetColorTransferFunction(v)
    vLUT.ApplyPreset('Black-Body Radiation', True)
    w.render()

varlist = [n for n in list(df.columns) if n not in posCols]
interact(setVar, v=varlist)